In [ ]:
# importing necessary modules

import pandas as pd
import numpy as np

import os
for dirname, _, filenames in os.walk('dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
def merge_csv(path_to_csv1: str, path_to_csv2: str):
    data1 = pd.read_csv(path_to_csv1)
    data2 = pd.read_csv(path_to_csv2)

    # Union of columns
    columns = list(set(data1.columns | data2.columns))

    d = {col: data1[col] if col in data1.columns else data2[col] for col in columns}
    merged_data = pd.DataFrame(data=d)

    # Rearranging columns
    new_order = ['DY', 'MO', 'YEAR', 'LAT', 'LON']
    columns_list = merged_data.columns.tolist()
    for col_name in new_order:
        columns_list.remove(col_name)

    columns_list = new_order + columns_list
    merged_data = merged_data[columns_list]

    return merged_data

def organize_data(merged_data : pd.DataFrame):
    organized_data = merged_data.copy()
    # If data has KT (Insolation Clearness Index) column it must be converted to float
    if 'KT' in merged_data:
        # 'KT' column may have 2 different values that describes missing values. nan and -999.0
        organized_data['KT'] = organized_data['KT'].replace('               nan', '-999')
        organized_data['KT'] = organized_data['KT'].astype(float)


    # Value for missing model data cannot be computed or out of model availability range: -999.0
    # Replace these values to np.nan in order to use pre-defined functions effectively.
    organized_data.replace(-999.0, np.nan, inplace=True)

    return organized_data

In [ ]:
df = organize_data(merge_csv("dataset\csv1.csv", "dataset\csv2.csv"))

### View dimensions of dataset

In [ ]:
df.shape

### Preview the dataset

In [ ]:
df.head()

### View column names

In [ ]:
df.columns

### View summary of dataset 

In [ ]:
df.info()

### View statistical properties of dataset

In [ ]:
df.describe()

In [ ]:
today_precipitation = df['PRECTOT']
tomorrow_precipitation = [1 if x > 0 else 0 for x in today_precipitation]
tomorrow_precipitation.pop(0)
tomorrow_precipitation.append(1)

df['PRECTOT_TODAY'] = today_precipitation
df['PRECTOT_TOMORROW'] = np.array(tomorrow_precipitation)
df.drop(columns=['LAT', 'LON'], inplace=True)

In [ ]:
df.info()